In [1]:
import os
os.environ['HF_HOME'] = 'huggingface'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = 'True'
import math
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Union
import evaluate

/home/alc/venvs/til2023/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'openai/whisper-small.en'
checkpoint_name = 'whisper-checkpoints/checkpoint-750/'

In [3]:
processor = WhisperProcessor.from_pretrained(model_name)

In [4]:
ds = load_dataset('audiofolder', data_dir='TIL_data_folder', split='train')  # specify split to return a Dataset object instead of a DatasetDict

Resolving data files: 100%|██████████| 3750/3750 [00:00<00:00, 141412.81it/s]
Found cached dataset audiofolder (/home/alc/TIL-2023/ASR/huggingface/datasets/audiofolder/default-111e26b5add33029/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


In [5]:
ds = ds.train_test_split(test_size=0.2)

In [6]:
def prepare_dataset(batch):
    model_name = 'openai/whisper-small.en'
    from transformers import WhisperProcessor
    processor = WhisperProcessor.from_pretrained(model_name)
    batch["input_features"] =[processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0] for audio in batch["audio"]]
    batch["input_length"] = [len(b) for b in batch["input_features"]]
    batch["labels"] = processor(text=batch["annotation"]).input_ids
    batch['length'] = batch["input_length"]
    return batch

ds = ds.map(prepare_dataset, num_proc=8, batched=True, batch_size=512)

In [7]:
# purpose of the data collator is to ensure that the inputs and labels are padded correctly

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [8]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [9]:
model = WhisperForConditionalGeneration.from_pretrained(
    model_name,  # checkpoint_name
    pad_token_id=processor.tokenizer.pad_token_id,
    mask_time_prob=0.5,  # 0.05
    mask_time_length=10, # 10
    mask_feature_prob=0.5, # 0
    mask_feature_length=10, # 10
    apply_spec_augment=True
)

In [10]:
model.freeze_encoder()

In [11]:
per_gpu_bs = 3
effective_bs = 32
training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-checkpoints",
    overwrite_output_dir =True,
    per_device_train_batch_size=per_gpu_bs,
    gradient_accumulation_steps=math.ceil(effective_bs/per_gpu_bs),
    learning_rate=1e-4,
    num_train_epochs=20,
    gradient_checkpointing=False,
    fp16=True,
    # bf16=True,  # for A100
    fp16_full_eval=True,
    # bf16_full_eval=True,  # for A100
    group_by_length=True,  # slows down
    evaluation_strategy="epoch",
    save_strategy='epoch',  # epoch
    save_safetensors=True,
    per_device_eval_batch_size=4,
    save_steps=1,
    eval_steps=1,
    logging_steps=100,
    save_total_limit=3,
    lr_scheduler_type='cosine',
    load_best_model_at_end=True,  # True
    adam_beta1=0.9,
    adam_beta2=0.98,  # follow fairseq fintuning config
    warmup_ratio=0.22, # follow Ranger21
    weight_decay=1e-4,  # follow Ranger21
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to=['tensorboard'],
    dataloader_num_workers=24 if os.name != 'nt' else 1)

In [12]:
class CustomWhisperTrainer(Seq2SeqTrainer):
    def training_step(self, model: torch.nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)
        loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if os.name != 'nt':
            accelerator.backward(self.scaler.scale(loss))
            # self.scaler.scale(loss).backward()
        else:
            self.scaler.scale(loss).backward()
        return loss.detach()

In [13]:
if os.name != 'nt':
    from accelerate import Accelerator
    accelerator = Accelerator(mixed_precision='fp16', dynamo_backend='eager')  # FP8 needs transformer_engine package which is only on Linux with Hopper GPUs

In [34]:
def tri_stage_schedule(epoch: int, max_epoch = training_args.num_train_epochs, stage_ratio = [0.1, 0.4, 0.5], peak_lr = training_args.learning_rate, initial_lr_scale=0.01, final_lr_scale=0.05):
    """https://github.com/facebookresearch/fairseq/blob/5ecbbf58d6e80b917340bcbf9d7bdbb539f0f92b/fairseq/optim/lr_scheduler/tri_stage_lr_scheduler.py#L51"""
    assert sum(stage_ratio) == 1
    current_ratio = epoch / max_epoch
    if current_ratio < stage_ratio[0]:  # linear warmup
        lrs = torch.linspace(initial_lr_scale * peak_lr, peak_lr, int(stage_ratio[0] * max_epoch))
        return lrs[epoch]
    elif stage_ratio[0] <= current_ratio <= stage_ratio[1]:  # constant
        return peak_lr
    else:  # exponential decay
        decay_factor = -math.log(final_lr_scale) / (stage_ratio[2] * max_epoch)
        return peak_lr * math.exp(-decay_factor * stage_ratio[2] * max_epoch)

In [15]:
# max_steps = math.ceil(training_args.num_train_epochs * len(ds['train']) / training_args.gradient_accumulation_steps / min(training_args.per_device_train_batch_size, len(ds['train'])))
# optimizer = Ranger21(model.parameters(), num_iterations=max_steps, lr=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-8, foreach=False)  # https://github.com/facebookresearch/fairseq/blob/main/examples/wav2vec/config/finetuning/base_960h.yaml
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_steps)
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=tri_stage_schedule)  # following FAIR finetuning settings
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda x: x)  # constant LR, stays same throughout, for Ranger21
# torch._dynamo.config.suppress_errors = True
# torch._dynamo.config.verbose=True
trainer = CustomWhisperTrainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # optimizers=(optimizer, scheduler),
)
if os.name != 'nt':  # windows does not support torch.compile yet
    trainer.model_wrapped, trainer.optimizer, trainer.lr_scheduler = accelerator.prepare(trainer.model_wrapped, trainer.optimizer, trainer.lr_scheduler)
trainer.train()
if os.name != 'nt':
    accelerator.wait_for_everyone()

/home/alc/venvs/til2023/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/alc/venvs/til2023/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[2023-05-21 01:32:33,666] torch._dynamo.convert_frame: [WARNING] torch._dynamo hit config.cache_size_limit (64)
   function: 'forward' (/home/alc/venvs/til2023/lib/python3.10/site-packages

Epoch,Training Loss,Validation Loss


/home/alc/venvs/til2023/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


OutOfMemoryError: CUDA out of memory. Tried to allocate 916.00 MiB (GPU 0; 23.69 GiB total capacity; 19.54 GiB already allocated; 859.00 MiB free; 21.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
if os.name != 'nt':
    trainer.model_wrapped = accelerator.unwrap_model(trainer.model_wrapped)
trainer.save_model('wav2vec2-conformer')
processor.tokenizer.save_pretrained('wav2vec2-conformer')

In [2]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained('wav2vec2-conformer')
processor.tokenizer.save_pretrained('wav2vec2-checkpoints/checkpoint-2116/')

('checkpoints/checkpoint-4125/tokenizer_config.json',
 'checkpoints/checkpoint-4125/special_tokens_map.json',
 'checkpoints/checkpoint-4125/vocab.json',
 'checkpoints/checkpoint-4125/added_tokens.json')

In [1]:
# Infer
import os
os.environ['HF_HOME'] = 'huggingface'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = 'True'
import torch
import datasets
from transformers import Wav2Vec2Processor, Wav2Vec2ConformerForCTC
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import pandas as pd
from torch.utils.data import DataLoader
torch.multiprocessing.set_start_method('spawn')

/home/alc/venvs/til2023/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset("test", split="train")
dataset = KeyDataset(KeyDataset(dataset, "audio"), "array")
test_ds = pd.read_csv('Test_Advanced.csv')

Resolving data files: 100%|██████████| 12000/12000 [00:00<00:00, 26477.21it/s]
Found cached dataset audiofolder (/home/alc/TIL-2023/ASR/huggingface/datasets/audiofolder/test-b719a705ed310f32/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


In [3]:
def clean(annotation):
    if "'" in annotation:
        # print(annotation, f'has \' in {annotation}, removing')
        annotation = annotation.split("'")[0] + annotation.split("'")[1][1:]  # Tokenizer includes "'" but TIL dataset does not, remove the S following '
    return annotation

def collate_fn(batch):
    input_values = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)
    return {"input_values": input_values}

In [4]:
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-conformer")
data_loader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn, pin_memory=True, num_workers=4 if os.name == 'nt' else 0)
checkpoint1 = 'checkpoints/checkpoint-4125'
# checkpoint2 = 'checkpoints/checkpoint-11250'
model1 = Wav2Vec2ConformerForCTC.from_pretrained(checkpoint1).to('cuda')
# model2 = Wav2Vec2ConformerForCTC.from_pretrained(checkpoint2).to('cuda')
model1.eval()
# model2.eval()
logits1 = []
# logits2 = []
logits = []
with torch.no_grad():
    for batch in tqdm(data_loader):
        inputs = batch['input_values'].to('cuda')
        outputs1 = model1(**inputs).logits
        # outputs2 = model2(**inputs).logits
        logits1.append(outputs1)
        # logits2.append(outputs2)

100%|██████████| 375/375 [02:53<00:00,  2.16it/s]


In [8]:
# logits = [(l1 + l2) / 2 for l1, l2 in zip(logits1, logits2)]
results = []
for l in logits1:
    results.extend(processor.batch_decode(torch.argmax(l, dim=-1)))

In [9]:
test_ds['annotation'] = list(map(clean,results))
test_ds['path'] = test_ds['path'].apply(lambda x: x.split('/')[-1])
test_ds.to_csv('Test_Advanced zerui aug nomusan_0.00276.csv', index=False)  # change file name